In [ ]:
import os
import torch
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import random

from botorch.models import FixedNoiseGP
from botorch.models.model_list_gp_regression import ModelListGP
from botorch.models.transforms.outcome import Standardize
from gpytorch.mlls.sum_marginal_log_likelihood import SumMarginalLogLikelihood
from botorch.acquisition.multi_objective.monte_carlo import qExpectedHypervolumeImprovement
from botorch.utils.transforms import normalize, unnormalize
from botorch.utils.multi_objective.hypervolume import Hypervolume
from botorch.sampling.normal import SobolQMCNormalSampler
from botorch.utils.multi_objective.box_decompositions.non_dominated import NondominatedPartitioning
from botorch.utils.multi_objective.pareto import is_non_dominated
from botorch.utils.testing import BotorchTestCase

import matplotlib.pyplot as plt
import seaborn as sns
from botorch.utils.multi_objective import is_non_dominated
from sklearn.decomposition import PCA

PLOTS_DIR = "plots"
os.makedirs(PLOTS_DIR, exist_ok=True)
print(f"✅ Created directory '{PLOTS_DIR}/' for saving plots.")
sns.set_theme(style="whitegrid")

def plot_pareto_front(train_Y, iteration_counter, initial_size, new_points=None, save_path=None):
    fig, ax = plt.subplots(1, 1, figsize=(8, 7))
    
    evaluated_data = -train_Y.cpu().numpy()
    from botorch.utils.multi_objective.pareto import is_non_dominated
    is_pareto = is_non_dominated(train_Y).cpu().numpy()
    
    ax.scatter(evaluated_data[:, 0], evaluated_data[:, 1], c='silver', alpha=0.4, s=30, label='Evaluated')
    
    ax.scatter(evaluated_data[:initial_size, 0], evaluated_data[:initial_size, 1], 
               c='white', edgecolors='#1f77b4', linewidth=1.5, s=60, label='Initial Set')
    
    pareto_points = evaluated_data[is_pareto]
    sort_idx = np.argsort(pareto_points[:, 0])
    ax.step(pareto_points[sort_idx, 0], pareto_points[sort_idx, 1], where='post', c='red', alpha=0.3, linestyle='--')
    ax.scatter(pareto_points[:, 0], pareto_points[:, 1], c='#d62728', s=120, marker='*', edgecolors='black', linewidth=0.5, label='Pareto Front', zorder=10)
    
    if new_points is not None:
        new_data = -new_points.cpu().numpy()
        ax.scatter(new_data[:, 0], new_data[:, 1], c='#2ca02c', s=100, marker='P', edgecolors='black', label='New Candidates', zorder=11)
    
    ax.set_xlabel(r'Reaction Free Energy $\Delta G$ (kcal/mol)', fontsize=13)
    ax.set_ylabel(r'Activation Free Energy $\Delta G^{\ddagger}$ (kcal/mol)', fontsize=13)
    ax.set_title(f"Pareto Optimization: Iteration {iteration_counter - 1}", fontsize=15, fontweight='bold')
    ax.legend(frameon=True, loc='upper right', fontsize=10)
    ax.grid(True, linestyle=':', alpha=0.6)
    
    if save_path:
        plt.savefig(save_path, dpi=300, bbox_inches='tight')
    plt.show()
    plt.close(fig)

def plot_acquisition_function_large_scale(ehvi_values, next_candidate_indices, save_path=None):
    fig, ax = plt.subplots(figsize=(10, 5))
    
    x_pos = np.arange(len(ehvi_values))
    ax.fill_between(x_pos, ehvi_values, color='skyblue', alpha=0.4)
    ax.plot(x_pos, ehvi_values, color='#1f77b4', linewidth=0.8, alpha=0.7)
    
    selected_values = ehvi_values[next_candidate_indices]
    ax.vlines(next_candidate_indices, 0, selected_values, color='red', linestyle='-', alpha=0.6)
    ax.scatter(next_candidate_indices, selected_values, color='red', s=40, zorder=5, label='Selected for Batch')
    
    ax.set_xlabel("Candidate Index", fontsize=12)
    ax.set_ylabel(r"$\alpha_{EHVI}(\mathbf{x})$", fontsize=14)
    ax.set_title("Expected Hypervolume Improvement Distribution", fontsize=14)
    ax.grid(axis='y', linestyle='--', alpha=0.5)
    ax.legend()
    
    if save_path:
        plt.savefig(save_path, dpi=300, bbox_inches='tight')
    plt.show()
    plt.close(fig)

def plot_feature_space(X_pool, train_X, initial_size, new_X=None, save_path=None):
    if X_pool.shape[1] < 2: return
        
    pca = PCA(n_components=2)
    X_pool_2d = pca.fit_transform(X_pool.cpu().numpy())
    train_X_2d = pca.transform(train_X.cpu().numpy())
    
    fig, ax = plt.subplots(figsize=(8, 7))

    ax.scatter(X_pool_2d[:, 0], X_pool_2d[:, 1], c='whitesmoke', s=20, edgecolors='gainsboro', label='Chemical Space')
    
    ax.scatter(train_X_2d[:initial_size, 0], train_X_2d[:initial_size, 1], 
               c='#1f77b4', s=50, alpha=0.7, label='Initial Points')
    
    end_idx = len(train_X_2d) - (len(new_X) if new_X is not None else 0)
    ax.scatter(train_X_2d[initial_size:end_idx, 0], train_X_2d[initial_size:end_idx, 1], 
               c='#ff7f0e', s=50, alpha=0.7, label='BO Iterations')
    
    if new_X is not None:
        new_X_2d = pca.transform(new_X.cpu().numpy())
        ax.scatter(new_X_2d[:, 0], new_X_2d[:, 1], c='#2ca02c', s=120, marker='P', edgecolors='black', label='Next Batch')
    
    ax.set_xlabel(f"PC1 ({pca.explained_variance_ratio_[0]:.1%} var.)", fontsize=12)
    ax.set_ylabel(f"PC2 ({pca.explained_variance_ratio_[1]:.1%} var.)", fontsize=12)
    ax.set_title("Exploration in Latent Feature Space", fontsize=14)
    ax.legend(loc='best', frameon=True, shadow=True)
    
    if save_path:
        plt.savefig(save_path, dpi=300, bbox_inches='tight')
    plt.show()
    plt.close(fig)

def plot_gp_regression(model, train_X, train_Y, standardizer, bounds, objective_names, save_path=None):
    model.eval()
    latex_names = [r"$\Delta G$", r"$\Delta G^{\ddagger}$"]
    
    normalized_train_X = normalize(train_X, bounds)
    with torch.no_grad():
        posterior = model.posterior(normalized_train_X)
    
    mean_pred_unstandardized = standardizer.untransform(posterior.mean)[0]
    y_pred_orig = -mean_pred_unstandardized.cpu().numpy()
    y_true_orig = -train_Y.cpu().numpy()
    
    M = train_Y.shape[-1]
    fig, axes = plt.subplots(1, M, figsize=(12, 5))
    if M == 1: axes = [axes]
    
    for i in range(M):
        ax = axes[i]
        y_t, y_p = y_true_orig[:, i], y_pred_orig[:, i]
        
        lims = [min(y_t.min(), y_p.min())-1, max(y_t.max(), y_p.max())+1]
        ax.plot(lims, lims, 'k--', alpha=0.5, zorder=0)

        rmse = np.sqrt(np.mean((y_t - y_p)**2))
        ax.scatter(y_t, y_p, c='#17becf', edgecolors='navy', alpha=0.6, s=40, label=f'RMSE: {rmse:.2f}')
        
        ax.set_xlabel(f"True {latex_names[i]} (kcal/mol)", fontsize=12)
        ax.set_ylabel(f"GP Predicted {latex_names[i]}", fontsize=12)
        ax.set_title(f"Model Fidelity: {latex_names[i]}", fontsize=14)
        ax.legend()
        ax.set_aspect('equal')
        ax.grid(True, linestyle=':', alpha=0.5)

    plt.tight_layout()
    if save_path:
        plt.savefig(save_path, dpi=300, bbox_inches='tight')
    plt.show()
    plt.close(fig)

def plot_hypervolume_evolution(hypervolume_history, initial_hv, save_path=None):
    fig, ax = plt.subplots(figsize=(9, 6))
    
    all_hvs = [initial_hv] + hypervolume_history
    x_pos = np.arange(len(all_hvs))
    
    ax.fill_between(x_pos, all_hvs, initial_hv, color='lavender', alpha=0.5)
    ax.plot(x_pos, all_hvs, marker='o', markersize=8, linewidth=2, color='#58508d', markerfacecolor='#ffa600', label='Observed HV')
    
    ax.set_xlabel("Optimization Step (Batch Index)", fontsize=12)
    ax.set_ylabel("Hypervolume (HV)", fontsize=12)
    ax.set_title("Optimization Convergence: Hypervolume Progression", fontsize=14)
    
    ax.set_xticks(x_pos)
    labels = ['Init'] + [str(i) for i in range(1, len(all_hvs))]
    ax.set_xticklabels(labels)
    
    ax.grid(axis='y', linestyle='--', alpha=0.7)
    ax.legend(loc='lower right')
    
    if save_path:
        plt.savefig(save_path, dpi=300, bbox_inches='tight')
    plt.show()
    plt.close(fig)

SEED = 12345 
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(SEED)
    
# BotorchTestCase.seed(SEED)
print(f"✅ Global random seed set to: {SEED}")

tkwargs = {
    "dtype": torch.double,
    "device": torch.device("cuda" if torch.cuda.is_available() else "cpu"),
}

print(f"Using device: {tkwargs['device']}")

try:
    df_pool = pd.read_csv("../../descriptors_cleaned.csv")
except FileNotFoundError:
    print("Error: Please ensure your data file is named 'descriptors_cleaned.csv' and is located in the same folder as the script.")
    
CANDIDATE_NAMES_POOL = df_pool.iloc[:, 0].tolist()
X_CANDIDATES_POOL = torch.tensor(df_pool.iloc[:, 1:].values, **tkwargs)
FEATURE_COLUMNS = df_pool.columns[1:]

D = X_CANDIDATES_POOL.shape[1]
M = 2
BOUNDS = torch.stack([X_CANDIDATES_POOL.min(dim=0).values, X_CANDIDATES_POOL.max(dim=0).values])
print(f"Successfully loaded {len(CANDIDATE_NAMES_POOL)} candidate molecules with {D} descriptors each.")
print("\n--- Performing Data Cleaning ---")

features_df = df_pool.iloc[:, 1:]
nan_columns = features_df.columns[features_df.isnull().any()].tolist()

constant_columns = features_df.columns[features_df.nunique() == 1].tolist()

columns_to_drop = list(set(nan_columns + constant_columns))
if columns_to_drop:
    print("\n⚠️ Found the following problematic feature columns to be removed:")
    if nan_columns:
        print(f"  - {len(nan_columns)} columns contain NaN values: {nan_columns}")
    if constant_columns:
        print(f"  - {len(constant_columns)} columns are constant: {constant_columns}")

    df_pool = df_pool.drop(columns=columns_to_drop)

    print(f"\n✅ Removed all problematic features. Remaining feature count: {df_pool.shape[1] - 1}")
else:
    print("\n✅ Dataset is clean with no constant or NaN-containing feature columns.")

rows_to_remove_mask = df_pool['name'].str.contains('frame_6')

num_to_remove = rows_to_remove_mask.sum()
original_rows = len(df_pool)
print(f"Initial DataFrame has {original_rows} rows.")
print(f"Identified {num_to_remove} rows with 'name' containing 'frame_6' to be removed.")

df_pool_cleaned = df_pool[~rows_to_remove_mask]
final_rows = len(df_pool_cleaned)

print(f"Operation completed. The new DataFrame has {final_rows} rows.")
print(f"Successfully removed {original_rows - final_rows} rows.")
df_pool = df_pool_cleaned

X_CANDIDATES_POOL = torch.tensor(df_pool.iloc[:, 1:].values, **tkwargs)
FEATURE_COLUMNS = df_pool.columns[1:]
D = X_CANDIDATES_POOL.shape[1]
BOUNDS = torch.stack([X_CANDIDATES_POOL.min(dim=0).values, X_CANDIDATES_POOL.max(dim=0).values])
print(f"Data cleaning completed. Updated feature dimension: D={D}")

N_INITIAL = 5
INITIAL_CSV_PATH = "initial_candidates.csv"

In [ ]:

df_initial_results = pd.read_csv(INITIAL_CSV_PATH)
df_initial = pd.read_csv(INITIAL_CSV_PATH)
INITIAL_TRAIN_SIZE = len(df_initial)


if df_initial_results[['dG', 'dG_ts']].isnull().values.any():
    print("🛑 Error: 'initial_candidates.csv' still contains unfilled data. Please complete the calculations and fill in the data before running this cell again.")
else:
    train_Y = torch.tensor(-df_initial_results[['dG', 'dG_ts']].values, **tkwargs)

    train_X_list = []
    for name in df_initial_results['name']:
        idx = CANDIDATE_NAMES_POOL.index(name)
        train_X_list.append(X_CANDIDATES_POOL[idx])
    train_X = torch.stack(train_X_list)

    evaluated_names = set(df_initial_results['name'].tolist())

    iteration_counter = 1

    print("\n✅ Initial data loaded successfully!")
    print(f"Current training set size: {len(train_X)}")

ref_point_hv = train_Y.min(dim=0).values - 0.2
hv_calculator = Hypervolume(ref_point=ref_point_hv)
initial_hypervolume = hv_calculator.compute(train_Y)
hypervolume_history = [] 
print(f"\n✅ Initial hypervolume calculation completed: {initial_hypervolume:.4f}")
print(f"   (using reference point: {ref_point_hv.cpu().numpy()})")

In [ ]:
BATCH_SIZE = 5

print(f"\n--- Iteration {iteration_counter}: Suggesting next batch of {BATCH_SIZE} candidate points ---")

standardizer = Standardize(m=M)
train_Y_standardized, _ = standardizer(train_Y)

models = []
for j in range(M):
    train_y_j_standardized_col = train_Y_standardized[:, j].unsqueeze(-1)
    models.append(
        FixedNoiseGP(
            train_X=normalize(train_X, BOUNDS),
            train_Y=train_y_j_standardized_col,
            train_Yvar=torch.full_like(train_y_j_standardized_col, 1e-6)

        )
    )
model = ModelListGP(*models)
mll = SumMarginalLogLikelihood(model.likelihood, model)
from botorch.fit import fit_gpytorch_mll
fit_gpytorch_mll(mll)
print("\n--- Plotting current GP model regression performance ---")
plot_gp_regression(
    model=model,
    train_X=train_X,
    train_Y=train_Y,
    standardizer=standardizer,
    bounds=BOUNDS,
    objective_names=['dG', 'dG_ts'],
    save_path=f"{PLOTS_DIR}/gp_regression_iter_{iteration_counter}.png"
)

ref_point = train_Y.min(dim=0).values - 0.2
acq_func = qExpectedHypervolumeImprovement(
    model=model,
    ref_point=ref_point,
    partitioning=NondominatedPartitioning(ref_point=ref_point, Y=train_Y_standardized),
    sampler=SobolQMCNormalSampler(sample_shape=torch.Size([128])),
)

unseen_mask = ~df_pool['name'].isin(evaluated_names)
unseen_df = df_pool[unseen_mask]
unseen_indices = unseen_df.index.tolist()
unseen_X_candidates = X_CANDIDATES_POOL[unseen_indices]

if len(unseen_df) < BATCH_SIZE:
    print("\n⚠️ Warning: The number of remaining candidate molecules is less than the batch size.")
    current_batch_size = len(unseen_df)
else:
    current_batch_size = BATCH_SIZE
print(f"\n--- Iteration {iteration_counter}: Evaluating EHVI for {len(unseen_df)} remaining candidate molecules ---")

normalized_unseen_X = normalize(unseen_X_candidates, BOUNDS)
ACQ_EVAL_BATCH_SIZE = 1024 
ehvi_values_list = []
candidate_batches = torch.split(normalized_unseen_X, ACQ_EVAL_BATCH_SIZE)

print(f"\n--- Splitting into {len(candidate_batches)} batches of size {ACQ_EVAL_BATCH_SIZE} for EHVI evaluation to save memory ---")
with torch.no_grad():
    for i, X_batch in enumerate(candidate_batches):
        print(f"  - Now processing batch {i+1}/{len(candidate_batches)}...")
        ehvi_values_list.append(acq_func(X_batch.unsqueeze(1)))
        
ehvi_values = torch.cat(ehvi_values_list)

_, top_local_indices = torch.topk(ehvi_values, k=current_batch_size) 

plot_acquisition_function_large_scale(
    ehvi_values=ehvi_values.cpu().numpy(),
    next_candidate_indices=top_local_indices.cpu().numpy(),
    save_path=f"{PLOTS_DIR}/ehvi_dist_iter_{iteration_counter}.png"
)
top_original_indices = [unseen_indices[i] for i in top_local_indices]
next_candidates_df = df_pool.iloc[top_original_indices].copy()

next_candidates_df['dG'] = np.nan
next_candidates_df['dG_ts'] = np.nan
NEXT_CSV_PATH = f"iteration_{iteration_counter}_candidates_batch.csv"
next_candidates_df.to_csv(NEXT_CSV_PATH, index=False)

print(f"\n🎉 Successfully suggested the next batch of {current_batch_size} candidate molecules.")
print(f"Task saved to file: '{NEXT_CSV_PATH}'")
print("Suggested molecule list:")
print(next_candidates_df[['name']])




In [ ]:

if not os.path.exists(NEXT_CSV_PATH):
    print(f"🛑 File '{NEXT_CSV_PATH}' does not exist. Please run Cell 4 to generate the recommendation file first.")
else:
    df_new_batch = pd.read_csv(NEXT_CSV_PATH)

    df_success = df_new_batch.dropna(subset=['dG', 'dG_ts']).copy()

    all_names_in_batch = df_new_batch['name'].tolist()

    evaluated_names.update(all_names_in_batch)
    
    if df_success.empty:
        print("\n⚠️ No completed rows found in '{NEXT_CSV_PATH}'.")
        print("Have marked all recommended molecules as processed; they will not be recommended again.")
        print(f"Processed/failed molecules: {all_names_in_batch}")

    else:
        successful_names = df_success['name'].tolist()
        new_y = torch.tensor(-df_success[['dG', 'dG_ts']].values, **tkwargs)
        new_x = torch.tensor(df_success[FEATURE_COLUMNS].values, **tkwargs)
    
        train_X = torch.cat([train_X, new_x], dim=0)
        train_Y = torch.cat([train_Y, new_y], dim=0)
        
        print("\n✅ Successfully added new data points!")
        print(f"Molecule list: {successful_names}")
        
        failed_names = list(set(all_names_in_batch) - set(successful_names))
        if failed_names:
            print(f"ℹ️  {len(failed_names)} error or incomplete calculations have been recorded and will not be recommended again: {failed_names}")

    iteration_counter += 1


    print("-" * 30)
    print(f"Current training set size: {len(train_X)}")
    print(f"Total number of molecules processed/evaluated: {len(evaluated_names)}")
    print("\n👉 You can now go back and re-run 【Cell 4】 to recommend the next batch of candidates.")

    print("\n--- Plotting state after this iteration ---")

    new_y_to_plot = torch.tensor(-df_success[['dG', 'dG_ts']].values, **tkwargs) if not df_success.empty else None
    plot_pareto_front(
        train_Y=train_Y, 
        iteration_counter=iteration_counter,
        initial_size=INITIAL_TRAIN_SIZE,
        new_points=new_y_to_plot,
        save_path=f"{PLOTS_DIR}/pareto_front_iter_{iteration_counter - 1}.png"
    )
    
    new_x_to_plot = new_x if not df_success.empty else None
    plot_feature_space(
        X_pool=X_CANDIDATES_POOL,
        train_X=train_X,
        initial_size=INITIAL_TRAIN_SIZE,
        new_X=new_x_to_plot, 
        save_path=f"{PLOTS_DIR}/feature_space_iter_{iteration_counter - 1}.png"
    )
